#### 1° String de conexão (ajuste conforme seu ambiente)

In [12]:
from sqlalchemy import create_engine, text

server = 'DESKTOP-F0DI39E'
database = 'master'  # criaremos o banco a partir do master
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)

#### 2° (OPCIONAL) Criar a base de dados via Python

In [ ]:
# from sqlalchemy import Engine, text


# with Engine.connect() as conn:
#     conn.execute(text("""
#         IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'BI_Projeto')
#         CREATE DATABASE BI_Projeto;
#     """))
#     conn.commit()

#### 3° Conectar na base criada

In [13]:
database = 'Ourobase_ProjetoBI'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)

#### 4° Criação das tabelas dbo.Cliente

In [ ]:
create_table_sql = """
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Clientes'
)
BEGIN
    CREATE TABLE dbo.Clientes (
        
id_Cliente INT IDENTITY(1,1) PRIMARY KEY,
str_Cliente VARCHAR(150) NOT NULL,



id_Usuario INT IDENTITY(1,1) PRIMARY KEY,
str_Usuario VARCHAR(150) NULL,
str_Email VARCHAR(150) NOT NULL,
bit_StatusAtivo BIT NOT NULL,
str_DescrInativo VARCHAR(255) NULL,



id_Servidor INT IDENTITY(1,1) PRIMARY KEY,
str_TipoAcessoServidor VARCHAR(100) NOT NULL,
str_NomeServidor VARCHAR(150) NOT NULL,
str_NomeDataBase VARCHAR(150) NOT NULL,



id_ProjetoBI INT IDENTITY(1,1) PRIMARY KEY,
str_NomeProjetoBI VARCHAR(150) NOT NULL,
bit_PossuiAplicativo BIT NOT NULL,
str_TipoLicenca VARCHAR(100) NOT NULL,
bit_PossuiRegraNegocio BIT NULL,
str_DescrRegraRLSBI VARCHAR(255) NULL,
int_QtdeAtualizacoes INT NOT NULL,
str_AtualizacaoAgendada VARCHAR(100) NOT NULL,
bit_PossuiBIPersonalizado BIT NOT NULL,
str_DescrBIPersonalizado VARCHAR(255) NULL,
str_VersionamentoProjeto VARCHAR(50) NULL,



id_Melhoria INT IDENTITY(1,1) PRIMARY KEY,
bit_MelhoriasBI BIT NOT NULL,
str_DescrMelhoriasBI VARCHAR(255) NULL,
dta_DataMelhoria DATE NULL,



id_Pendencia INT IDENTITY(1,1) PRIMARY KEY,
int_NroPendencia INT NULL,
dta_DataInicioPendencia DATE NULL,
dta_DataFimPendencia DATE NULL,



id_Atendimento INT IDENTITY(1,1) PRIMARY KEY,
int_NroAtendimento INT NULL,
dta_DataInicioAtendimento DATE NULL,
dta_DataFimAtendimento DATE NULL

    );
END
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxe incorreta próxima a 'id_Usuario'. (102) (SQLExecDirectW)")
[SQL: 
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Clientes'
)
BEGIN
    CREATE TABLE dbo.Clientes (

id_Cliente INT IDENTITY(1,1) PRIMARY KEY,
str_Cliente VARCHAR(150) NOT NULL



id_Usuario INT IDENTITY(1,1) PRIMARY KEY,
str_Usuario VARCHAR(150) NULL,
str_Email VARCHAR(150) NOT NULL,
bit_StatusAtivo BIT NOT NULL,
str_DescrInativo VARCHAR(255) NULL,



id_Servidor INT IDENTITY(1,1) PRIMARY KEY,
str_TipoAcessoServidor VARCHAR(100) NOT NULL,
str_NomeServidor VARCHAR(150) NOT NULL,
str_NomeDataBase VARCHAR(150) NOT NULL,



id_ProjetoBI INT IDENTITY(1,1) PRIMARY KEY,
str_NomeProjetoBI VARCHAR(150) NOT NULL,
bit_PossuiAplicativo BIT NOT NULL,
str_TipoLicenca VARCHAR(100) NOT NULL,
bit_PossuiRegraNegocio BIT NULL,
str_DescrRegraRLSBI VARCHAR(255) NULL,
int_QtdeAtualizacoes INT NOT NULL

#### 5°Inserir dados via Python dbo.Cliente

In [ ]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=SEU_SERVIDOR;"
    "DATABASE=SEU_BANCO;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

# Truncate
cursor.execute("TRUNCATE TABLE dim_Cliente")
conn.commit()

# Load Excel
df = pd.read_excel("base_projetos.xlsx", sheet_name="Cliente")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO dim_Cliente (str_Cliente)
        VALUES (?)
    """, row['str_Cliente'])

conn.commit()
cursor.close()
conn.close()


#### 6° (Opcional) Consultar os dados

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dbo.Cliente"))
    for row in result:
        print(row)

### 7° OPICIONAL Atualizar com SQLAlchemy (caso você esteja usando)

In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(
    "mssql+pyodbc://@localhost/BI_Projeto?driver=ODBC+Driver+17+for+SQL+Server"
)

with engine.connect() as conn:
    result = conn.execute(
        text("""
            UPDATE dbo.Cliente
            SET str_Cliente = :nome
            WHERE id_Cliente = :id
        """),
        {"nome": "001 - singular medicamentos", "id": 2}
    )
    conn.commit()